# Deep Learning for Content-Based Filtering

Implement content-based filtering using a neural network to build a recommender system for movies. 


# Outline
- [ 1 - Packages ](#1)
- [ 2 - Movie ratings dataset ](#2)
- [ 3 - Content-based filtering with a neural network](#3)
  - [ 3.1 Training Data](#3.1)
  - [ 3.2 Preparing the training data](#3.2)
- [ 4 - Neural Network for content-based filtering](#4)
- [ 5 - Predictions](#5)
  - [ 5.1 - Predictions for a new user](#5.1)
  - [ 5.2 - Predictions for an existing user.](#5.2)
  - [ 5.3 - Finding Similar Items](#5.3)
- [ 6 - Conclusion](#6)


<a name="1"></a>
## 1 - Packages 
Kita akan menggunakan beberapa library yang familiar seperti NumPy, TensorFlow, scikit-learn, tabulate, dan Pandas. Library-library ini membantu kita dalam mengelola dan memproses data.

In [76]:
# Mengimpor Library yang diperlukan
import numpy as np
import numpy.ma as ma
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import tabulate
pd.set_option("display.precision", 1)

<a name="2"></a>
## 2 - Movie ratings dataset

Dataset yang digunakan berasal dari [MovieLens ml-latest-small](https://grouplens.org/datasets/movielens/latest/)

Awalnya, dataset ini memiliki hampir 9000 film yang dinilai oleh 600 pengguna dengan skala penilaian 0.5 hingga 5. Untuk keperluan analisis, dataset telah diperkecil fokus pada film-film yang dirilis sejak tahun 2000 dan film-film populer. Versi yang lebih kecil ini mencakup 397 pengguna, 847 film, dan 25,521 penilaian.

Setiap film dalam dataset mencakup informasi tentang judul, tanggal rilis, dan satu atau lebih genre. Misalnya, "Toy Story 3" memiliki genre "Adventure|Animation|Children|Comedy|Fantasy".

Berikut adalah 10 film teratas berdasarkan jumlah penilaian dan rata-rata penilaiannya.

In [77]:
# Membaca data dari file CSV
top10_df = pd.read_csv("./data/content_top10_df.csv")
bygenre_df = pd.read_csv("./data/content_bygenre_df.csv")

# Menampilkan DataFrame top10_df
top10_df


,movie id,num ratings,ave rating,title,genres
0,4993,198,4.1,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy
1,5952,188,4.0,"Lord of the Rings: The Two Towers, The",Adventure|Fantasy
2,7153,185,4.1,"Lord of the Rings: The Return of the King, The",Action|Adventure|Drama|Fantasy
3,4306,170,3.9,Shrek,Adventure|Animation|Children|Comedy|Fantasy|Ro...
4,58559,149,4.2,"Dark Knight, The",Action|Crime|Drama
5,6539,149,3.8,Pirates of the Caribbean: The Curse of the Bla...,Action|Adventure|Comedy|Fantasy
6,79132,143,4.1,Inception,Action|Crime|Drama|Mystery|Sci-Fi|Thriller
7,6377,141,4.0,Finding Nemo,Adventure|Animation|Children|Comedy
8,4886,132,3.9,"Monsters, Inc.",Adventure|Animation|Children|Comedy|Fantasy
9,7361,131,4.2,Eternal Sunshine of the Spotless Mind,Drama|Romance|Sci-Fi


Tabel berikut menunjukkan data yang diurutkan berdasarkan genre. Jumlah penilaian untuk setiap genre bisa sangat bervariasi. Ingat, satu film bisa memiliki beberapa genre, jadi jumlah penilaiannya bisa lebih besar dari jumlah penilaian aslinya.

In [78]:
# Menampilkan DataFrame bygenre_df
bygenre_df

,genre,num movies,ave rating/genre,ratings per genre
0,Action,321,3.4,10377
1,Adventure,234,3.4,8785
2,Animation,76,3.6,2588
3,Children,69,3.4,2472
4,Comedy,326,3.4,8911
5,Crime,139,3.5,4671
6,Documentary,13,3.8,280
7,Drama,342,3.6,10201
8,Fantasy,124,3.4,4468
9,Horror,56,3.2,1345


<a name="3"></a>
## 3 - Content-based filtering with a neural network

Dalam metode pemfilteran berbasis konten, kita akan membuat representasi vektor fitur untuk setiap pengguna dan film. Dengan menambahkan informasi tambahan ke dalam neural network, kita bisa menghasilkan representasi yang lebih baik dan memberikan prediksi yang lebih akurat.

<a name="3.1"></a>
### 3.1 Training Data
Data pelatihan terdiri dari dua komponen utama:
- Fitur Film: Kombinasi dari data asli (seperti tahun rilis dan genre) serta fitur yang dihasilkan (seperti rating rata-rata).
- Fitur Pengguna: Fitur yang dihasilkan seperti rata-rata rating per genre untuk setiap pengguna.


In [79]:
# Memuat data dan mengatur variabel konfigurasi
item_train, user_train, y_train, item_features, user_features, item_vecs, movie_dict, user_to_genre = load_data()

num_user_features = user_train.shape[1] - 3  # Menghapus userid, jumlah rating, dan rata-rata rating selama pelatihan
num_item_features = item_train.shape[1] - 1  # Menghapus movie id selama pelatihan
uvs = 3  # Mulai dari kolom genre pengguna
ivs = 3  # Mulai dari kolom genre item
u_s = 3  # Mulai dari kolom ke-3 untuk pelatihan, pengguna
i_s = 1  # Mulai dari kolom pertama untuk pelatihan, item
print(f"Number of training vectors: {len(item_train)}")

Number of training vectors: 50884


Memeriksa beberapa entri pertama dalam array pelatihan pengguna dan film untuk memahami representasi data tersebut

In [80]:
#Data Pengguna
pprint_train(user_train, user_features, uvs,  u_s, maxcount=5)

[user id],[rating count],[rating ave],Act ion,Adve nture,Anim ation,Chil dren,Com edy,Crime,Docum entary,Drama,Fan tasy,Hor ror,Mys tery,Rom ance,Sci -Fi,Thri ller
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9


In [81]:
#Data Film
pprint_train(item_train, item_features, ivs, i_s, maxcount=5, user=False)

[movie id],year,ave rating,Act ion,Adve nture,Anim ation,Chil dren,Com edy,Crime,Docum entary,Drama,Fan tasy,Hor ror,Mys tery,Rom ance,Sci -Fi,Thri ller
6874,2003,4.0,1,0,0,0,0,1,0,0,0,0,0,0,0,1
8798,2004,3.8,1,0,0,0,0,1,0,1,0,0,0,0,0,1
46970,2006,3.2,1,0,0,0,1,0,0,0,0,0,0,0,0,0
48516,2006,4.3,0,0,0,0,0,1,0,1,0,0,0,0,0,1
58559,2008,4.2,1,0,0,0,0,1,0,1,0,0,0,0,0,0


Langkah selanjutnya adalah memeriksa data target yang merupakan rating film yang diberikan oleh pengguna.

In [82]:
print(f"y_train[:5]: {y_train[:5]}")

y_train[:5]: [4.  3.5 4.  4.  4.5]


<a name="3.2"></a>
### 3.2 Preparing the training data
Melakukan scaling untuk meningkatkan kinerja model. Kita akan menggunakan StandardScaler dari scikit-learn untuk fitur input dan MinMaxScaler untuk target rating, mengubahnya ke rentang antara -1 dan 1.

In [83]:
# Melakukan scaling pada data pelatihan

# Menyimpan data pelatihan asli yang belum diskalakan
item_train_unscaled = item_train
user_train_unscaled = user_train
y_train_unscaled    = y_train

# Scaling data item menggunakan StandardScaler
scalerItem = StandardScaler()  # Membuat objek StandardScaler untuk item
scalerItem.fit(item_train)  # Menghitung rata-rata dan standar deviasi untuk scaling
item_train = scalerItem.transform(item_train)  # Menerapkan scaling pada data item

# Scaling data pengguna menggunakan StandardScaler
scalerUser = StandardScaler()  # Membuat objek StandardScaler untuk pengguna
scalerUser.fit(user_train)  # Menghitung rata-rata dan standar deviasi untuk scaling
user_train = scalerUser.transform(user_train)  # Menerapkan scaling pada data pengguna

# Scaling target rating menggunakan MinMaxScaler dengan rentang -1 hingga 1
scalerTarget = MinMaxScaler(feature_range=(-1, 1))  # Membuat objek MinMaxScaler
scalerTarget.fit(y_train.reshape(-1, 1))  # Menghitung minimum dan maksimum untuk scaling
y_train = scalerTarget.transform(y_train.reshape(-1, 1))  # Menerapkan scaling pada target rating

# Mengembalikan data yang telah diskalakan ke bentuk aslinya untuk verifikasi
print(np.allclose(item_train_unscaled, scalerItem.inverse_transform(item_train)))  # Memeriksa apakah data item yang diskalakan bisa dikembalikan ke bentuk asli
print(np.allclose(user_train_unscaled, scalerUser.inverse_transform(user_train)))  # Memeriksa apakah data pengguna yang diskalakan bisa dikembalikan ke bentuk asli


True
True


Untuk mengevaluasi model, kita akan membagi data menjadi set pelatihan dan pengujian menggunakan train_test_split dari scikit-learn.

In [84]:
# Membagi data menjadi set pelatihan dan pengujian
item_train, item_test = train_test_split(item_train, train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(user_train, train_size=0.80, shuffle=True, random_state=1)
# Membagi data target rating menjadi set pelatihan dan pengujian
y_train, y_test       = train_test_split(y_train,    train_size=0.80, shuffle=True, random_state=1)
# Membagi data target rating menjadi set pelatihan dan pengujian
print(f"movie/item training data shape: {item_train.shape}")
print(f"movie/item test data shape: {item_test.shape}")

movie/item training data shape: (40707, 17)
movie/item test data shape: (10177, 17)


Data yang telah diskalakan dan diacak sekarang siap digunakan. Kita akan memeriksa beberapa entri pertama dalam data pelatihan untuk memastikan semuanya benar.

In [85]:
pprint_train(user_train, user_features, uvs, u_s, maxcount=5)

[user id],[rating count],[rating ave],Act ion,Adve nture,Anim ation,Chil dren,Com edy,Crime,Docum entary,Drama,Fan tasy,Hor ror,Mys tery,Rom ance,Sci -Fi,Thri ller
1,0,-1.0,-0.8,-0.7,0.1,-0.0,-1.2,-0.4,0.6,-0.5,-0.5,-0.1,-0.6,-0.6,-0.7,-0.7
0,1,-0.7,-0.5,-0.7,-0.1,-0.2,-0.6,-0.2,0.7,-0.5,-0.8,0.1,-0.0,-0.6,-0.5,-0.4
-1,-1,-0.2,0.3,-0.4,0.4,0.5,1.0,0.6,-1.2,-0.3,-0.6,-2.3,-0.1,0.0,0.4,-0.0
0,-1,0.6,0.5,0.5,0.2,0.6,-0.1,0.5,-1.2,0.9,1.2,-2.3,-0.1,0.0,0.2,0.3
-1,0,0.7,0.6,0.5,0.3,0.5,0.4,0.6,1.0,0.6,0.3,0.8,0.8,0.4,0.7,0.7


<a name="4"></a>
## 4 - Neural Network for content-based filtering
Membangun jaringan neural dengan dua sub-jaringan yang digabungkan oleh dot product. Kedua jaringan akan identik dalam contoh ini, dan kita akan menggunakan Keras Sequential dengan spesifikasi berikut:
- Lapisan pertama: dense layer dengan 256 unit dan aktivasi relu.
- Lapisan kedua: dense layer dengan 128 unit dan aktivasi relu.
- Lapisan ketiga: dense layer dengan num_outputs unit dan aktivasi linear atau tanpa aktivasi.


In [86]:
num_outputs = 32 # Menentukan jumlah output
tf.random.set_seed(1) # Mengatur seed untuk memastikan hasil yang dapat direproduksi

# Membangun jaringan neural untuk pengguna menggunakan Keras Sequential API
user_NN = tf.keras.models.Sequential([  
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_outputs, activation='linear'),
])

# Membangun jaringan neural untuk item menggunakan Keras Sequential API
item_NN = tf.keras.models.Sequential([   
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_outputs, activation='linear'),
])

# Membuat input layer untuk pengguna dan menghubungkannya dengan jaringan neural pengguna
input_user = tf.keras.layers.Input(shape=(num_user_features))
vu = user_NN(input_user)
vu = tf.linalg.l2_normalize(vu, axis=1)

# Membuat input layer untuk item dan menghubungkannya dengan jaringan neural item
input_item = tf.keras.layers.Input(shape=(num_item_features))
vm = item_NN(input_item)
vm = tf.linalg.l2_normalize(vm, axis=1)

# Menghitung dot product dari dua vektor vu dan vm
output = tf.keras.layers.Dot(axes=1)([vu, vm])

# Menentukan input dan output dari model
model = tf.keras.Model([input_user, input_item], output)

model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 14)]         0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            [(None, 16)]         0                                            
__________________________________________________________________________________________________
sequential_4 (Sequential)       (None, 32)           40864       input_8[0][0]                    
__________________________________________________________________________________________________
sequential_5 (Sequential)       (None, 32)           41376       input_9[0][0]                    
____________________________________________________________________________________________

Mengkompilasi model menggunakan fungsi loss Mean Squared Error dan optimizer Adam.

In [87]:
tf.random.set_seed(1)
cost_fn = tf.keras.losses.MeanSquaredError()
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt,
              loss=cost_fn)

Melatih model menggunakan data pelatihan yang telah disiapkan sebelumnya. Model akan dilatih selama 30 epoch.

In [88]:
tf.random.set_seed(1)
model.fit([user_train[:, u_s:], item_train[:, i_s:]], y_train, epochs=30)

Train on 40707 samples
Epoch 1/30
40707/40707 [==============================] - 5s 128us/sample - loss: 0.1232
Epoch 2/30
40707/40707 [==============================] - 5s 120us/sample - loss: 0.1146
Epoch 3/30
40707/40707 [==============================] - 5s 120us/sample - loss: 0.1089
Epoch 4/30
40707/40707 [==============================] - 5s 118us/sample - loss: 0.1039
Epoch 5/30
40707/40707 [==============================] - 5s 120us/sample - loss: 0.1001
Epoch 6/30
40707/40707 [==============================] - 5s 120us/sample - loss: 0.0973
Epoch 7/30
40707/40707 [==============================] - 5s 120us/sample - loss: 0.0956
Epoch 8/30
40707/40707 [==============================] - 5s 120us/sample - loss: 0.0935
Epoch 9/30
40707/40707 [==============================] - 5s 119us/sample - loss: 0.0916
Epoch 10/30
40707/40707 [==============================] - 5s 120us/sample - loss: 0.0897
Epoch 11/30
40707/40707 [==============================] - 5s 118us/sample - loss: 0.0

Mengevaluasi kinerja model pada data pengujian untuk melihat seberapa baik model tersebut dalam memprediksi data yang belum pernah dilihat selama pelatihan.

In [89]:
model.evaluate([user_test[:, u_s:], item_test[:, i_s:]], y_test)

10177/10177 [==============================] - 0s 39us/sample - loss: 0.0815


0.08146006993124337

<a name="5"></a>
## 5 - Predictions
Menggunakan model yang telah dilatih untuk membuat prediksi dalam berbagai situasi. 
<a name="5.1"></a>
### 5.1 - Predictions for a new user
Kita akan menggunakan model untuk merekomendasikan film bagi pengguna baru berdasarkan preferensi mereka.

In [90]:
#Definisikan Preferensi Pengguna Baru: Misalkan pengguna baru ini menyukai film dengan genre petualangan dan fantasi.
new_user_id = 5000
new_rating_ave = 0.0
new_action = 0.0
new_adventure = 5.0
new_animation = 0.0
new_childrens = 0.0
new_comedy = 0.0
new_crime = 0.0
new_documentary = 0.0
new_drama = 0.0
new_fantasy = 5.0
new_horror = 0.0
new_mystery = 0.0
new_romance = 0.0
new_scifi = 0.0
new_thriller = 0.0
new_rating_count = 3

user_vec = np.array([[new_user_id, new_rating_count, new_rating_ave,
                      new_action, new_adventure, new_animation, new_childrens,
                      new_comedy, new_crime, new_documentary,
                      new_drama, new_fantasy, new_horror, new_mystery,
                      new_romance, new_scifi, new_thriller]])

Buat Prediksi untuk Pengguna Baru:
- Replikasikan vektor pengguna baru untuk mencocokkan jumlah film dalam dataset.
- Skalakan vektor pengguna dan item.
- Lakukan prediksi menggunakan model.
- Urutkan hasil prediksi, dari rating tertinggi.

In [91]:
# Menghasilkan dan mereplikasi vektor pengguna agar sesuai dengan jumlah film dalam dataset
user_vecs = gen_user_vecs(user_vec, len(item_vecs))

# Melakukan scaling pada vektor pengguna dan item
suser_vecs = scalerUser.transform(user_vecs)
sitem_vecs = scalerItem.transform(item_vecs)

# Membuat prediksi menggunakan model
y_p = model.predict([suser_vecs[:, u_s:], sitem_vecs[:, i_s:]])

# Mengembalikan skala prediksi y ke nilai aslinya
y_pu = scalerTarget.inverse_transform(y_p)

# Mengurutkan hasil prediksi, prediksi tertinggi di urutan pertama
sorted_index = np.argsort(-y_pu, axis=0).reshape(-1).tolist()  # Menggunakan negatif untuk mendapatkan rating tertinggi terlebih dahulu
sorted_ypu = y_pu[sorted_index]
sorted_items = item_vecs[sorted_index]  # Menggunakan vektor yang tidak diskalakan untuk tampilan

# Menampilkan prediksi film
print_pred_movies(sorted_ypu, sorted_items, movie_dict, maxcount=10)


y_p,movie id,rating ave,title,genres
4.5,98809,3.8,"Hobbit: An Unexpected Journey, The (2012)",Adventure|Fantasy
4.4,8368,3.9,Harry Potter and the Prisoner of Azkaban (2004),Adventure|Fantasy
4.4,54001,3.9,Harry Potter and the Order of the Phoenix (2007),Adventure|Drama|Fantasy
4.3,40815,3.8,Harry Potter and the Goblet of Fire (2005),Adventure|Fantasy|Thriller
4.3,106489,3.6,"Hobbit: The Desolation of Smaug, The (2013)",Adventure|Fantasy
4.3,81834,4,Harry Potter and the Deathly Hallows: Part 1 (2010),Action|Adventure|Fantasy
4.3,59387,4,"Fall, The (2006)",Adventure|Drama|Fantasy
4.3,5952,4,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy
4.3,5816,3.6,Harry Potter and the Chamber of Secrets (2002),Adventure|Fantasy
4.3,54259,3.6,Stardust (2007),Adventure|Comedy|Fantasy|Romance


<a name="5.2"></a>
### 5.2 - Predictions for an existing user.
Kita akan melihat prediksi untuk pengguna yang sudah ada dalam dataset. Kita dapat membandingkan rating yang diprediksi oleh model dengan rating aktual pengguna tersebut.

In [92]:
uid = 2 
# Membentuk satu set vektor pengguna. Vektor ini sama, ditransformasikan dan diulang.
user_vecs, y_vecs = get_user_vecs(uid, user_train_unscaled, item_vecs, user_to_genre)

# Melakukan scaling pada vektor pengguna dan item
suser_vecs = scalerUser.transform(user_vecs)
sitem_vecs = scalerItem.transform(item_vecs)

# Membuat prediksi menggunakan model
y_p = model.predict([suser_vecs[:, u_s:], sitem_vecs[:, i_s:]])

# Mengembalikan skala prediksi y ke nilai aslinya
y_pu = scalerTarget.inverse_transform(y_p)

# Mengurutkan hasil prediksi, prediksi tertinggi di urutan pertama
sorted_index = np.argsort(-y_pu, axis=0).reshape(-1).tolist()  # Menggunakan negatif untuk mendapatkan rating tertinggi terlebih dahulu
sorted_ypu = y_pu[sorted_index]
sorted_items = item_vecs[sorted_index]  # Menggunakan vektor yang tidak diskalakan untuk tampilan
sorted_user = user_vecs[sorted_index]
sorted_y = y_vecs[sorted_index]

# Menampilkan prediksi yang diurutkan untuk film yang diberi rating oleh pengguna
print_existing_user(sorted_ypu, sorted_y.reshape(-1, 1), sorted_user, sorted_items, ivs, uvs, movie_dict, maxcount=50)


y_p,y,user,user genre ave,movie rating ave,movie id,title,genres
4.5,5.0,2,[4.0],4.3,80906,Inside Job (2010),Documentary
4.2,3.5,2,"[4.0,4.0]",3.9,99114,Django Unchained (2012),Action|Drama
4.1,4.5,2,"[4.0,4.0]",4.1,68157,Inglourious Basterds (2009),Action|Drama
4.1,3.5,2,"[4.0,3.9,3.9]",3.9,115713,Ex Machina (2015),Drama|Sci-Fi|Thriller
4.0,4.0,2,"[4.0,4.1,4.0,4.0,3.9,3.9]",4.1,79132,Inception (2010),Action|Crime|Drama|Mystery|Sci-Fi|Thriller
4.0,4.0,2,"[4.1,4.0,3.9]",4.3,48516,"Departed, The (2006)",Crime|Drama|Thriller
4.0,4.5,2,"[4.0,4.1,4.0]",4.2,58559,"Dark Knight, The (2008)",Action|Crime|Drama
4.0,4.0,2,"[4.0,4.1,3.9]",4.0,6874,Kill Bill: Vol. 1 (2003),Action|Crime|Thriller
4.0,3.5,2,"[4.0,4.1,4.0,3.9]",3.8,8798,Collateral (2004),Action|Crime|Drama|Thriller
3.9,5.0,2,"[4.0,4.1,4.0]",3.9,106782,"Wolf of Wall Street, The (2013)",Comedy|Crime|Drama


<a name="5.3"></a>
### 5.3 - Finding Similar Items
Jaringan neural yang telah dibangun menghasilkan dua vektor fitur: vektor fitur pengguna (𝑣𝑢) dan vektor fitur film (𝑣𝑚). Vektor-vektor ini sulit diinterpretasikan secara langsung, tetapi item yang serupa akan memiliki vektor yang serupa. Informasi ini dapat digunakan untuk memberikan rekomendasi.

Ukuran keserupaan antara dua vektor dapat diukur menggunakan jarak kuadrat (squared distance) antara kedua vektor tersebut:

In [93]:
def sq_dist(a, b):
    """
    Mengembalikan jarak kuadrat antara dua vektor
    Args:
      a (ndarray (n,)): vektor dengan n fitur
      b (ndarray (n,)): vektor dengan n fitur
    Returns:
      d (float) : jarak kuadrat
    """    
    d = sum(np.square(a - b))  # Menghitung jarak kuadrat antara dua vektor
    return d


In [94]:
# Membuat beberapa vektor untuk pengujian fungsi sq_dist
a1 = np.array([1.0, 2.0, 3.0]); b1 = np.array([1.0, 2.0, 3.0])
a2 = np.array([1.1, 2.1, 3.1]); b2 = np.array([1.0, 2.0, 3.0])
a3 = np.array([0, 1, 0]);       b3 = np.array([1, 0, 0])
print(f"squared distance between a1 and b1: {sq_dist(a1, b1):0.3f}")
print(f"squared distance between a2 and b2: {sq_dist(a2, b2):0.3f}")
print(f"squared distance between a3 and b3: {sq_dist(a3, b3):0.3f}")

squared distance between a1 and b1: 0.000
squared distance between a2 and b2: 0.030
squared distance between a3 and b3: 2.000


Untuk membuat rekomendasi tanpa perlu melatih ulang model, kita bisa menghitung matriks jarak antara film-film. Langkah pertama adalah mendapatkan vektor fitur film (𝑣𝑚) untuk setiap film. Kita akan menggunakan jaringan neural terlatih item_NN untuk membangun model kecil yang memungkinkan kita menjalankan vektor film melalui jaringan untuk menghasilkan 𝑣𝑚.

In [95]:
input_item_m = tf.keras.layers.Input(shape=(num_item_features))    # Membuat input layer dengan shape sesuai jumlah fitur item
vm_m = item_NN(input_item_m)                                       # Menggunakan model item_NN yang telah dilatih untuk memproses input item
vm_m = tf.linalg.l2_normalize(vm_m, axis=1)                        # Normalisasi vektor fitur item untuk memastikan nilai-nilai berada dalam skala yang konsisten
model_m = tf.keras.Model(input_item_m, vm_m)                       # Membangun model dengan input dan output yang telah ditentukan         
model_m.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, 16)]         0                                            
__________________________________________________________________________________________________
sequential_5 (Sequential)       (None, 32)           41376       input_10[0][0]                   
__________________________________________________________________________________________________
tf_op_layer_l2_normalize_9/Squa [(None, 32)]         0           sequential_5[1][0]               
__________________________________________________________________________________________________
tf_op_layer_l2_normalize_9/Sum  [(None, 1)]          0           tf_op_layer_l2_normalize_9/Square
____________________________________________________________________________________________

Setelah model untuk memprediksi vektor fitur film dibuat, kita akan menggunakan model ini untuk menghitung vektor fitur film dan kemudian membuat matriks jarak kuadrat antar film.

In [96]:
scaled_item_vecs = scalerItem.transform(item_vecs)
vms = model_m.predict(scaled_item_vecs[:,i_s:])
print(f"size of all predicted movie feature vectors: {vms.shape}")

size of all predicted movie feature vectors: (847, 32)


Kita akan menghitung matriks jarak kuadrat antar film dan menampilkan film yang paling mirip berdasarkan vektor fitur yang telah diprediksi.

In [97]:
count = 50  # jumlah film yang akan ditampilkan
dim = len(vms)  # mendapatkan jumlah film
dist = np.zeros((dim, dim))  # inisialisasi matriks jarak dengan nilai nol

# Menghitung jarak kuadrat antara semua pasangan film
for i in range(dim):
    for j in range(dim):
        dist[i, j] = sq_dist(vms[i, :], vms[j, :])

# Masking diagonal matriks jarak untuk menghindari jarak nol dengan dirinya sendiri
m_dist = ma.masked_array(dist, mask=np.identity(dist.shape[0]))

# Inisialisasi tampilan data yang akan ditampilkan dalam tabel
disp = [["movie1", "genres", "movie2", "genres"]]

# Menemukan film yang paling mirip berdasarkan jarak kuadrat
for i in range(count):
    min_idx = np.argmin(m_dist[i])  # mendapatkan indeks film dengan jarak terpendek
    movie1_id = int(item_vecs[i, 0])  # mendapatkan ID film pertama
    movie2_id = int(item_vecs[min_idx, 0])  # mendapatkan ID film kedua
    disp.append([movie_dict[movie1_id]['title'], movie_dict[movie1_id]['genres'],
                 movie_dict[movie2_id]['title'], movie_dict[movie2_id]['genres']])

# Menampilkan hasil dalam format tabel HTML
table = tabulate.tabulate(disp, tablefmt='html', headers="firstrow")
table


movie1,genres,movie2,genres
Save the Last Dance (2001),Drama|Romance,Mona Lisa Smile (2003),Drama|Romance
"Wedding Planner, The (2001)",Comedy|Romance,Mr. Deeds (2002),Comedy|Romance
Hannibal (2001),Horror|Thriller,Final Destination 2 (2003),Horror|Thriller
Saving Silverman (Evil Woman) (2001),Comedy|Romance,Down with Love (2003),Comedy|Romance
Down to Earth (2001),Comedy|Fantasy|Romance,Bewitched (2005),Comedy|Fantasy|Romance
"Mexican, The (2001)",Action|Comedy,Rush Hour 2 (2001),Action|Comedy
15 Minutes (2001),Thriller,Panic Room (2002),Thriller
Enemy at the Gates (2001),Drama,Kung Fu Hustle (Gong fu) (2004),Action|Comedy
Heartbreakers (2001),Comedy|Crime|Romance,Fun with Dick and Jane (2005),Comedy|Crime
Spy Kids (2001),Action|Adventure|Children|Comedy,"Tuxedo, The (2002)",Action|Comedy


<a name="6"></a>
## 6 - Conclusion!
Kesamaan Film: Film dalam kolom "movie2" adalah film yang paling mirip dengan film dalam kolom "movie1" berdasarkan vektor fitur yang diprediksi oleh model. Kesamaan ini diukur menggunakan jarak kuadrat antara vektor fitur film.
Genre Serupa: Genre film dalam setiap pasangan sangat mirip atau identik. Ini menunjukkan bahwa model mampu menangkap kesamaan berdasarkan genre film yang merupakan salah satu fitur dalam vektor fitur.

Contoh Pasangan Film:

Pasangan 1:
Film 1: Save the Last Dance (2001)
Genre 1: Drama|Romance
Film 2: Mona Lisa Smile (2003)
Genre 2: Drama|Romance

Pasangan 2:
Film 1: Hannibal (2001)
Genre 1: Horror|Thriller
Film 2: Final Destination 2 (2003)
Genre 2: Horror|Thriller

Kesimpulan:
Dengan langkah-langkah ini, kita telah berhasil membangun, melatih, dan mengevaluasi model rekomendasi film berbasis konten yang menggunakan jaringan neural. Model ini mampu memberikan rekomendasi yang relevan dan akurat kepada pengguna, baik baru maupun yang sudah ada, serta menemukan film-film yang serupa berdasarkan fitur-fiturnya. Implementasi ini menunjukkan bagaimana deep learning dapat digunakan untuk meningkatkan sistem rekomendasi dan memberikan pengalaman yang lebih personal kepada pengguna.